## 完整案例

In [1]:
import ast
import base64
import os

import branca
import cv2
import folium
import pandas as pd
import requests
from folium.plugins import MarkerCluster
from IPython.display import IFrame
from PIL import Image
from PIL.ExifTags import TAGS

In [2]:
def findAllImage(rawImagePath, endsWith='.jpg'):
    """
    rawImagePath: 原图的文件夹路径
    返回所有图片列表
    """
    fileNames = os.listdir(rawImagePath)
    for fileName in fileNames:
        if not fileName.endswith(endsWith):
            fileNames.remove(fileName)
    return fileNames

In [3]:
def getGpsInfo(exifInfo):
    """
    exifInfo: 图片信息
    返回gps定位
    """
    for tag, gpsValue in list(exifInfo.items()):
        decoded = TAGS.get(tag)
        if decoded == 'GPSInfo':
            return gpsValue

def analysisGps(GpsInfo):
    """
    GpsInfo:gps定位,经纬度
    返回十进制的地理信息
    """
    Npos = sum([(item[0]/item[1])*(1.0/(60**i)) for i, item in enumerate(GpsInfo[2])])
    Epos = sum([(item[0]/item[1])*(1.0/(60**i)) for i, item in enumerate(GpsInfo[4])])
    return Npos, Epos


In [4]:
HEIGHT = 300
WIDTH = 500

def resizeImage(fileName, multiple=1.5):
    """
    fileName ：图片所在文件夹的路径
    返回重定义尺寸的图片
    """
    Height = int(HEIGHT*multiple)
    Width = int(WIDTH*multiple)
    
    img = cv2.imread(fileName)
    height, width = img.shape[:2]
    
    if height > width:
        imgMini = cv2.resize(img, (int(img.shape[1] * Height / img.shape[0]), Height))
        
    else:
        imgMini = cv2.resize(img, (Width, int(img.shape[0] * Width / img.shape[1])))

    return imgMini

def resizeAllImage(imagePath, imageSavePath):
    """
    imagePath:图片所在的文件夹路径
    imageSavePath:图片处理后所在的文件夹路径
    """
    fileNames = findAllImage(imagePath)
    
    if not os.path.exists(imageSavePath):
        os.mkdir(imageSavePath)
    
    for fileName in fileNames:
        miniImage = resizeImage(os.path.join(imagePath, fileName))
        cv2.imwrite(os.path.join(imageSavePath, fileName), miniImage)
        
        
resizeAllImage('./img/', './miniImage')

In [5]:
def show_html(map, mapName, isShow=True):
    """
    map: folium地图
    map：定义地图名称
    以html格式保存地图，并展示该html
    """
    map.save(mapName)
    if isShow:
        return IFrame(src=mapName, width=980, height=600)
    else:
        return None

In [6]:
fileNames = findAllImage(rawImagePath='./img/')
# 所有图片地理信息的列表
geoDatas = []
for imgName in fileNames:
    img=Image.open('./img/' + imgName)
    exifInfo = img._getexif()
    GpsInfo = getGpsInfo(exifInfo)
    geoData = analysisGps(GpsInfo)
    geoDatas.append(geoData)

In [7]:
# 设置图片作为html格式的长和宽
HTMLDemo_width = """
            <center><img src='data:image/jpeg;base64,{}' style='width: {}px'></center>
            """

HTMLDemo_height = """
            <center><img src='data:image/jpeg;base64,{}' style='height: {}px'></center>
            """

In [8]:
tile = [
    'http://wprd01.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=7',
    'http://wprd02.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=2&style=7',
    'http://wprd02.is.autonavi.com/appmaptile?x={x}&y={y}&z={z}&lang=zh_cn&size=1&scl=1&style=6',
]

In [17]:
import folium

m = folium.Map(location = [30,120],zoom_start = 5 ,tiles = 'Stamen', attr = "Stamen")

m

In [9]:
# 创建底图
map = folium.Map(
    location=[30.243463, 120.138724],
    zoom_start=6,
    tiles=tile[2],
    attr='高德地图')

# 插件：聚类
marker_cluster = MarkerCluster().add_to(map)

# 具有自定义图标和标记集群的绘图标记
for geoData, fileName in zip(geoDatas, fileNames):
    icon_path = './icon/足迹.png'
    icon = folium.features.CustomIcon(icon_image=icon_path, icon_size=(30, 30))

    imagePathName = os.path.join('./miniImage/', fileName)
    encoded = base64.b64encode(open(imagePathName, 'rb').read())
    height, width = cv2.imread(imagePathName).shape[:2]
    
    if height > width:
        html = HTMLDemo_height.format(encoded.decode('UTF-8'), str(HEIGHT))
        iframe = branca.element.IFrame(html=html, width=width*HEIGHT/height+10, height=HEIGHT+20)
    else:
        html = HTMLDemo_width.format(encoded.decode('UTF-8'), str(WIDTH))
        iframe = branca.element.IFrame(html=html, width=WIDTH+10, height=height*WIDTH/width+10)
        
    popup = folium.Popup(iframe, max_width=500)

    folium.Marker(location=[geoData[0], geoData[1]], icon=icon,
                  popup=popup).add_to(marker_cluster)

# 保存并展示图
show_html(map, 'base_map_2.html', isShow=True)